In [12]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from citipy import citipy
from datetime import datetime
import time
import requests
from config import weather_api_keys


In [13]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [14]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

80.54423823711588 -73.0115752484857
-34.36085625160512 93.49088155181408
-34.85131875459322 11.041270260582195
46.0330806454852 -148.09999305049962
54.850909571356425 114.76966409121667
72.33449960377914 97.64206525351443
-85.54770907039776 -75.08894004087394
87.89987021774684 -153.5379003106851
59.62491750093693 65.03220981146998
77.40014479227403 -135.3121269479165
-11.588921405220802 -22.547347386257115
70.29251358795685 -143.799053892934
28.560900081143146 -82.10628172099322
88.77171123048981 49.1977301251776
-29.09047815434944 -136.46674322632975
-74.2842298495018 -100.92481766690037
7.19706548929841 86.51514486649256
-55.80695749476502 162.55549943337792
30.376595482227188 -102.69294022398229
-31.358989986433187 20.354436897646764
-67.38115129035518 166.5379281231953
63.55399241920435 -91.89462518391805
-49.068616838870014 15.670701161194245
-26.75874515184966 152.49633650756084
34.2043884386413 -141.43867147427937
-39.6963574880534 -123.3738755324706
-30.92753265184234 -89.93921

In [15]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)


751

### Most of this is copied straight from WeatherPy.ipynb

In [16]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_keys

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
#lets not do this again.....

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]   
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | qaanaaq
Processing Record 2 of Set 1 | geraldton
Processing Record 3 of Set 1 | saldanha
Processing Record 4 of Set 1 | kodiak
Processing Record 5 of Set 1 | bagdarin
Processing Record 6 of Set 1 | khatanga
Processing Record 7 of Set 1 | ushuaia
Processing Record 8 of Set 1 | barrow
Processing Record 9 of Set 1 | uray
Processing Record 10 of Set 1 | tuktoyaktuk
Processing Record 11 of Set 1 | georgetown
Processing Record 12 of Set 1 | fairbanks
Processing Record 13 of Set 1 | clermont
Processing Record 14 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 15 of Set 1 | rikitea
Processing Record 16 of Set 1 | punta arenas
Processing Record 17 of Set 1 | kalmunai
Processing Record 18 of Set 1 | bluff
Processing Record 19 of Set 1 | pecos
Processing Record 20 of Set 1 | calvinia
Processing Record 21 of Set 1 | thompson
Processing Record 22 of Set 1 | hermanus
Processing Record 23

In [17]:
len(city_data)
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,Qaanaaq,77.4840,-69.3632,51.21,few clouds,67,13,4.50,GL
1,Geraldton,-28.7667,114.6000,55.81,overcast clouds,82,100,2.30,AU
2,Saldanha,-33.0117,17.9442,63.52,scattered clouds,45,28,7.25,ZA
3,Kodiak,57.7900,-152.4072,62.11,overcast clouds,72,100,8.05,US
4,Bagdarin,54.4333,113.6000,56.35,overcast clouds,97,100,3.51,RU
5,Khatanga,71.9667,102.5000,55.45,few clouds,66,23,5.01,RU
6,Ushuaia,-54.8000,-68.3000,40.66,few clouds,60,20,3.44,AR
7,Barrow,71.2906,-156.7887,41.02,overcast clouds,81,100,6.91,US
8,Uray,60.1247,64.7758,62.01,scattered clouds,74,38,5.88,RU
9,Tuktoyaktuk,69.4541,-133.0374,62.60,broken clouds,48,75,16.11,CA


In [18]:
columns = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_data_df = city_data_df[columns]
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Qaanaaq,GL,77.4840,-69.3632,51.21,67,13,4.50,few clouds
1,Geraldton,AU,-28.7667,114.6000,55.81,82,100,2.30,overcast clouds
2,Saldanha,ZA,-33.0117,17.9442,63.52,45,28,7.25,scattered clouds
3,Kodiak,US,57.7900,-152.4072,62.11,72,100,8.05,overcast clouds
4,Bagdarin,RU,54.4333,113.6000,56.35,97,100,3.51,overcast clouds


In [19]:
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [20]:
# Create the output File (CSV)
output_data_file = "WeatherPy_Database.csv"
# Export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")